In [ ]:
!pip install wandb -qqq

     |████████████████████████████████| 1.8 MB 31.7 MB/s 
     |████████████████████████████████| 181 kB 64.8 MB/s 
     |████████████████████████████████| 145 kB 72.7 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 


In [ ]:
!pip install wandb
!pip install wordcloud
!pip install colour

In [ ]:
## Installing font for Hindi for matplotlib ##
!apt-get install -y fonts-lohit-deva
!fc-list :lang=hi family

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following NEW packages will be installed:
  fonts-lohit-deva
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 78.2 kB of archives.
After this operation, 196 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 fonts-lohit-deva all 2.95.4-2 [78.2 kB]
Fetched 78.2 kB in 1s (63.4 kB/s)
Selecting previously unselected package fonts-lohit-deva.
(Reading database ... 155203 files and directories currently installed.)
Preparing to unpack .../fonts-lohit-deva_2.95.4-2_all.deb ...
Unpacking fonts-lohit-deva (2.95.4-2) ...
Setting up fonts-lohit-deva (2.95.4-2) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...
Lohit Devanagari


In [ ]:
!nvidia-smi


Fri May 13 13:40:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import matplotlib as mlp
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.font_manager import FontProperties
import numpy as np
import pandas as pd
import random
import time
import tensorflow as tf
from tensorflow import GradientTape
from tensorflow import keras
import datetime
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, GRU, SimpleRNN, SimpleRNNCell, LSTMCell, GRUCell
from keras.models import Sequential
from keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from sklearn.utils import shuffle
import wandb
import csv
from sklearn.metrics import confusion_matrix
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
from colour import Color
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!curl https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar --output dakshina_dataset_v1.0.tar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1915M  100 1915M    0     0  99.4M      0  0:00:19  0:00:19 --:--:--  105M


In [ ]:
!pwd


/content


In [ ]:
%%capture
!tar -xvf  'dakshina_dataset_v1.0.tar'

In [ ]:
!pwd

/content


In [ ]:
%cd '/content/dakshina_dataset_v1.0/'

/content/dakshina_dataset_v1.0


In [ ]:
!pwd

/content/dakshina_dataset_v1.0


In [ ]:
val_df = pd.read_csv("./hi/lexicons/hi.translit.sampled.dev.tsv", sep='\t', header=None)
# Number of training examples
M_train = len(val_df.keys())
train_df = pd.read_csv("./hi/lexicons/hi.translit.sampled.train.tsv", sep='\t', header=None)
# Number of validation examples
M_val = len(train_df.keys())
test_df = pd.read_csv("./hi/lexicons/hi.translit.sampled.test.tsv", sep='\t', header=None)
# Number of test examples
M_test = len(test_df.keys())

print(f"Number of training examples = {M_train}")
print(f"Number of validation examples = {M_val}")
print(f"Number of test examples = {M_test}")
print()
print("Data Loaded to Dataframes!")

Number of training examples = 3
Number of validation examples = 3
Number of test examples = 3

Data Loaded to Dataframes!


In [ ]:
%cd '/content/drive/My Drive/checkpoints/'

/content/drive/My Drive/checkpoints


In [ ]:
train_df.sample(n=3)

,0,1,2
26994,बांद्रा,bandra,3
5719,एबीएन,abn,3
42676,स्थायित्व,sthayitv,2


In [ ]:
sos = "@"
eos = "#"

In [ ]:
class LexDataset:
    def __init__(self, input_tensor, target_tensor, batch_size):
        self.input_tensor = input_tensor
        self.target_tensor = target_tensor
        self.batch = tf.data.Dataset.from_tensor_slices((self.input_tensor, self.target_tensor)).shuffle(len(self.input_tensor)).batch(batch_size, drop_remainder=True)

In [ ]:
class TransliterationDatatset:
    def __init__(self, df_list, batch_size = 64):
        
        self.input_tokenizer = None
        self.target_tokenizer = None
        self.train = None
        self.val = None
        self.test = None
        self.batch_size = batch_size
        # Load Data
        self.load_dataset(df_list)
        # Other parameters
        self.num_input_tokens = len(self.input_tokenizer.index_word)+1
        self.num_target_tokens = len(self.target_tokenizer.index_word)+1
        self.max_input_seq_length = np.max([self.train.input_tensor.shape[1], self.val.input_tensor.shape[1], self.test.input_tensor.shape[1]])
        self.max_target_seq_length = np.max([self.train.target_tensor.shape[1], self.val.target_tensor.shape[1], self.test.target_tensor.shape[1]])
        
    def preprocess_word(self, w):
        return sos + str(w) + eos
        
    def get_target_word(self, tensor):
        word = []
        for t in tensor:
            if t != 0:
                word.append(self.input_tokenizer.index_word[t])
        return "".join([ch for ch in word])
    
    def print_input(self, tensor):
        for t in tensor:
            if t != 0:
                print(f'{t} ----> {self.input_tokenizer.index_word[t]}')
    def print_target(self, tensor):
        for t in tensor:
            if t != 0:
                print(f'{t} ----> {self.target_tokenizer.index_word[t]}')

    def create_dataset(self, data_frame):
        input_words = []
        target_words = []
        # Shuffle the data_frame before creating dataset
        df = data_frame
        for i in range(5):
            df = shuffle(df)
        for x, y in zip(df[1], df[0]):
            input_words.append(self.preprocess_word(x))
            target_words.append(self.preprocess_word(y))
        return (input_words, target_words)

    
                
    
    
    
    
    def load_dataset(self, df_list):
        # df_list should have train -> val -> test in sequence
        
        self.input_tokenizer = Tokenizer(num_words = None, char_level = True)
        self.target_tokenizer = Tokenizer(num_words = None, char_level = True)
        
        ds_list = []
        
        for df in df_list:
            # Get the words list
            (input_words, target_words) = self.create_dataset(df)
            # Fit on the set of words
            self.input_tokenizer.fit_on_texts(input_words)
            self.target_tokenizer.fit_on_texts(target_words)
            ds_list.append((input_words, target_words))
                    
        self.target_tokenizer.index_word.update({0:" "})
        self.input_tokenizer.index_word.update({0:" "})
        
        input_word_len = []
        target_word_len = []
        
        tensor_list = []
        
        for i, (input_words, target_words) in enumerate(ds_list):
            input_tensor = self.input_tokenizer.texts_to_sequences(input_words)
            target_tensor = self.target_tokenizer.texts_to_sequences(target_words)
            tensor_list.append((input_tensor, target_tensor))
            input_word_len.append(np.max([len(x) for x in input_tensor]))
            target_word_len.append(np.max([len(x) for x in target_tensor]))
        
        for i, (input_tensor, target_tensor) in enumerate(tensor_list):
            
            input_tensor = pad_sequences(input_tensor, padding='post', maxlen = np.max(input_word_len))
            target_tensor = pad_sequences(target_tensor, padding='post', maxlen = np.max(target_word_len))
            
            if i == 0:
                self.train = LexDataset(input_tensor, target_tensor, self.batch_size)
            elif i == 1:
                self.val = LexDataset(input_tensor, target_tensor, self.batch_size)
            else:
                self.test = LexDataset(input_tensor, target_tensor, self.batch_size)

In [ ]:
dataset = TransliterationDatatset([train_df, val_df, test_df], 128)

In [ ]:
# Training data
dataset.train.input_tensor.shape, dataset.train.target_tensor.shape

((44204, 22), (44204, 21))

In [ ]:
# Validation data
dataset.val.input_tensor.shape, dataset.val.target_tensor.shape

((4358, 22), (4358, 21))

In [ ]:
# Test data
dataset.test.input_tensor.shape, dataset.test.target_tensor.shape

((4502, 22), (4502, 21))

In [ ]:
# Number of tokens
dataset.num_input_tokens, dataset.num_target_tokens

(30, 67)

In [ ]:
# max seq length
dataset.max_input_seq_length, dataset.max_target_seq_length

(22, 21)

In [ ]:
example_input_batch, example_target_batch = next(iter(dataset.train.batch))
example_input_batch.shape, example_target_batch.shape

(TensorShape([128, 22]), TensorShape([128, 21]))

In [ ]:
dataset.print_input(example_input_batch[2].numpy())

2 ----> @
21 ----> g
14 ----> l
1 ----> a
9 ----> s
9 ----> s
21 ----> g
11 ----> o
3 ----> #


In [ ]:
dataset.print_target(example_target_batch[2].numpy())

1 ----> @
23 ----> ग
5 ----> ्
13 ----> ल
3 ----> ा
9 ----> स
23 ----> ग
18 ----> ो
2 ----> #


In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz, dropout=0.2, layer_type="GRU", num_layers=1):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.layer_type = layer_type
        self.num_layers = num_layers
        self.dropout = dropout
        self.rnn_layers = []

        if self.layer_type == "LSTM":
            for i in range(self.num_layers):
                self.rnn_layers.append(LSTM(self.enc_units,return_sequences=True,return_state=True,dropout = self.dropout,recurrent_initializer='glorot_uniform'))
        
        elif self.layer_type == "GRU":
            for i in range(self.num_layers):
                self.rnn_layers.append(GRU(self.enc_units,return_sequences=True,return_state=True,dropout = self.dropout,recurrent_initializer='glorot_uniform'))
        
        else:
            for i in range(self.num_layers):
                self.rnn_layers.append(SimpleRNN(self.enc_units,return_sequences=True,return_state=True,dropout = self.dropout,recurrent_initializer='glorot_uniform'))        


    def call(self, inputs, hidden):
        inputs = self.embedding(inputs)
        state_h, state_c = [], []

        if self.layer_type == "LSTM":
            output, h, c = self.rnn_layers[0](inputs, initial_state = hidden)
            state_h.append(h)
            state_c.append(c)
            for i in range(1, self.num_layers):
                output, h, c = self.rnn_layers[i](output, initial_state = hidden)
                state_h.append(h)
                state_c.append(c)
            return output, state_h, state_c
        
        elif self.layer_type == "GRU":
            output, h = self.rnn_layers[0](inputs, initial_state = hidden)
            state_h.append(h)
            for i in range(1, self.num_layers):
                output, h = self.rnn_layers[i](output, initial_state = hidden)
                state_h.append(h)
            return output, state_h, None
           
        else:
            output, h = self.rnn_layers[0](inputs, initial_state = hidden)
            state_h.append(h)
            for i in range(1, self.num_layers):
                output, h = self.rnn_layers[i](output, initial_state = hidden)
                state_h.append(h)
            return output, state_h, None

    def initialize_hidden_state(self):
        if self.layer_type == "LSTM":
            return [tf.zeros((self.batch_sz, self.enc_units)), tf.zeros((self.batch_sz, self.enc_units))]
        else:
            return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
vocab_inp_size = dataset.num_input_tokens
embedding_dim = 64
units = 256
BATCH_SIZE = dataset.batch_size

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE, 0.2, "GRU", 3)

# # sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden, sample_cell = encoder(example_input_batch, sample_hidden)
print('Encoder output shape: (batch size, sequence length, units)', np.shape(sample_output))
print('Encoder Hidden state shape: (batch size, units)', np.shape(sample_hidden))
if encoder.layer_type == "LSTM":
  print ('Encoder c vector shape: (batch size, units) {}'.format(np.shape(sample_cell)))

Encoder output shape: (batch size, sequence length, units) (128, 22, 256)
Encoder Hidden state shape: (batch size, units) (3, 128, 256)


In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, dropout=0.2, layer_type="GRU", num_layers=1):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.layer_type = layer_type
        self.num_layers = num_layers
        self.dropout = dropout
        self.rnn_layers = []
        
        if self.layer_type == "LSTM":
            for i in range(self.num_layers):
                self.rnn_layers.append(LSTM(self.dec_units,return_sequences=True,return_state=True,dropout = self.dropout,recurrent_initializer='glorot_uniform'))
        
        elif self.layer_type == "GRU":
            for i in range(self.num_layers):
                self.rnn_layers.append(GRU(self.dec_units,return_sequences=True,return_state=True,dropout = self.dropout,recurrent_initializer='glorot_uniform'))
           
        else:
            for i in range(self.num_layers):
                self.rnn_layers.append(SimpleRNN(self.dec_units,return_sequences=True,return_state=True,dropout = self.dropout,recurrent_initializer='glorot_uniform'))
        
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, state_h, state_c=None):
        inputs = self.embedding(inputs)

        if self.layer_type == "LSTM":
            output, h, c = self.rnn_layers[0](inputs, initial_state = [state_h[0], state_c[0]])
            for i in range(1, self.num_layers):
                output, h, c = self.rnn_layers[i](output, initial_state = [state_h[i], state_c[i]])
        
        elif self.layer_type == "GRU":
            output, h = self.rnn_layers[0](inputs, initial_state = state_h[0])
            for i in range(1, self.num_layers):
                output, h = self.rnn_layers[i](output, initial_state = state_h[i])
           
        else:
            output, h = self.rnn_layers[0](inputs, initial_state = state_h[0])
            for i in range(1, self.num_layers):
                output, h = self.rnn_layers[i](output, initial_state = state_h[i])

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        # return x, state
        if self.layer_type != "LSTM":
            return x, h, None
        else:
            return x, h, c

In [ ]:
vocab_tar_size = dataset.num_target_tokens

In [ ]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, 0.2, "GRU", 3)
if decoder.layer_type != "LSTM":
  sample_decoder_output, sample_decoder_hidden, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)), sample_hidden, sample_cell)
else:
  sample_decoder_output, sample_decoder_hidden, sample_decoder_cell = decoder(tf.random.uniform((BATCH_SIZE, 1)), sample_hidden, sample_cell)

print('Decoder output shape: (batch_size, vocab size)', sample_decoder_output.shape)
print('Decoder Hidden state shape: (batch size, units)', sample_decoder_hidden.shape)
if encoder.layer_type == "LSTM":
  print ('Encoder c vector shape: (batch size, units) {}'.format(sample_decoder_cell.shape))

Decoder output shape: (batch_size, vocab size) (128, 67)
Decoder Hidden state shape: (batch size, units) (128, 256)


In [ ]:
def loss_function(real, pred, loss_object):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [ ]:
def accuracy(real, pred):
    real = tf.cast(real, tf.int32)
    pred = tf.cast(pred, tf.int32)
    return tf.reduce_mean(tf.cast(tf.equal(real, pred), tf.float32))

In [ ]:
def train_one_step():
    @tf.function
    def train_step(encoder, decoder, inp, targ, enc_hidden, optimizer, loss_object, is_val = False):
        loss = 0
        spc_loss = 0
        
        with tf.GradientTape() as tape:
            enc_output, enc_hidden, enc_cell = encoder(inp, enc_hidden)
            if decoder.layer_type != "LSTM":
                dec_hidden = enc_hidden
            else:
                dec_hidden, dec_cell = enc_hidden, enc_cell

            dec_input = tf.expand_dims([dataset.target_tokenizer.word_index[sos]] * dataset.batch_size, 1)

            pred = None

            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                
                # passing enc_output to the decoder
                if decoder.layer_type != "LSTM":
                    predictions, _, _ = decoder(dec_input, dec_hidden)
                else:
                    predictions, _, _ = decoder(dec_input, dec_hidden, dec_cell)
                            
                loss += loss_function(targ[:, t], predictions, loss_object)

                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)

                if t == 1:
                    pred = tf.expand_dims(tf.argmax(predictions, axis=-1), 1)
                else:
                    pred = tf.concat([pred, tf.expand_dims(tf.argmax(predictions, axis=-1), 1)], 1)
        
        batch_accuracy = accuracy(targ[:, 1:], pred)

        batch_loss = (loss / int(targ.shape[1]))

        if not is_val:
            variables = encoder.trainable_variables + decoder.trainable_variables

            gradients = tape.gradient(loss, variables)

            optimizer.apply_gradients(zip(gradients, variables))

        return batch_loss, batch_accuracy
    return train_step

In [ ]:
default_config = {
    "layer_type": "GRU",
    "units": 256,
    "embedding_dim": 64,
    "optimiser": "nadam",
    "epochs": 20,
    "dropout": 0.1,
    "batch_size": dataset.batch_size,
    "num_layers": 2
}

In [ ]:
def log_wandb(data):
    wandb.log(data)

In [ ]:
sweep_config = {
    "name": "Without Attention run" + str(datetime.datetime.now().replace(microsecond=0).isoformat()),
    "method": "random",
    "metric":{
        "name": "loss",
        "goal": "minimize"
    },
    "project": 'Assignment 3',
    "parameters": {
        "layer_type": {
            "values": ["GRU", "LSTM", "SimpleRNN"]
        },
        "dropout": {
            "values": [0.0, 0.1]
        },
        "units": {
            "values": [64, 256]
        },
        "embedding_dim": {
            "values": [16, 64]
        },
        "optimiser": {
            "values": ["nadam"]
        },
        "epochs": {
            "values": [20]
        },
        "batch_size": {
            "values": [dataset.batch_size]
        },
        "num_layers": {
            "values": [1, 2]
        }
    }
}

In [ ]:
def train(dataset, config, callback=None):

    run_name = "".join(f"{a}:{b} " for (a, b) in config.items())
    print(run_name)
    # wandb.run.name = run_name

    train_dataset = dataset.train
    val_dataset = dataset.val

    optimizer = tf.keras.optimizers.Nadam()
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

    EPOCHS = config["epochs"]
    BATCH_SIZE = config["batch_size"]
    steps_per_epoch = len(train_dataset.input_tensor)//BATCH_SIZE
    val_steps_per_epoch = len(val_dataset.input_tensor)//BATCH_SIZE
    embedding_dim = config["embedding_dim"]     
    units = config["units"]
    layer_type = config["layer_type"]
    num_layers = config["num_layers"]
    dropout = config["dropout"]

    # Encoder
    encoder = Encoder(dataset.num_input_tokens, embedding_dim, units, BATCH_SIZE, dropout, layer_type, num_layers)
    # Decoder
    decoder = Decoder(dataset.num_target_tokens, embedding_dim, units, BATCH_SIZE, dropout, layer_type, num_layers)

    train_step = train_one_step()

    for epoch in range(EPOCHS):
        start = time.time()

        enc_hidden = encoder.initialize_hidden_state()
        total_loss = 0
        total_accuracy = 0
        val_total_loss = 0
        val_total_accuracy = 0

        train_dataset.batch.shuffle(BATCH_SIZE*10)

        for (batch, (inp, targ)) in enumerate(train_dataset.batch.take(steps_per_epoch)):
            # Step Train
            batch_loss, batch_accuracy = train_step(encoder, decoder, inp, targ, enc_hidden, optimizer, loss_object, False)
            total_loss += batch_loss
            total_accuracy += batch_accuracy
            if batch % 100 == 0 or batch == steps_per_epoch-1:
                print(f'Epoch {epoch+1} Batch {batch} Loss {batch_loss.numpy():.4f} Accuracy {batch_accuracy:.4f}')
            
            if callback != None:
                callback({"epoch":epoch+1, "loss": batch_loss.numpy(), "accuracy":batch_accuracy})

        print(f'Epoch {epoch+1} Loss {total_loss/steps_per_epoch:.4f} Acc {total_accuracy/steps_per_epoch:.4f}')
        print(f'Time taken for 1 epoch {time.time()-start:.2f} sec\n')
        if callback != None:
            callback({"ep_training_loss": total_loss/steps_per_epoch, "ep_training_accuracy": total_accuracy/steps_per_epoch})

        val_dataset.batch.shuffle(BATCH_SIZE*10)

        for (batch, (inp, targ)) in enumerate(val_dataset.batch.take(val_steps_per_epoch)):
            val_batch_loss, val_batch_accuracy = train_step(encoder, decoder, inp, targ, enc_hidden, optimizer, loss_object, True)
            val_total_loss += val_batch_loss
            val_total_accuracy += val_batch_accuracy

            if batch % 100 == 0 or batch == val_steps_per_epoch-1:
                print(f'Epoch {epoch+1} Batch {batch} Val Loss {val_batch_loss.numpy():.4f} Val Accuracy {val_batch_accuracy:.4f}')

            if callback != None:
                callback({"epoch":epoch+1, "val loss": batch_loss.numpy(), "val accuracy":batch_accuracy})

        print(f'Epoch {epoch+1} Val Loss {val_total_loss/val_steps_per_epoch:.4f} Val Acc {val_total_accuracy/val_steps_per_epoch:.4f}')
        print(f'Time taken for 1 epoch {time.time()-start:.2f} sec\n')

        if callback != None:
            callback({"ep_val_loss": val_total_loss/val_steps_per_epoch, "ep_val_accuracy": val_total_accuracy/val_steps_per_epoch})

    return encoder, decoder

In [ ]:
enc, dec = train(dataset, default_config)

layer_type:GRU units:256 embedding_dim:64 optimiser:nadam epochs:20 dropout:0.1 batch_size:128 num_layers:2 
Epoch 1 Batch 0 Loss 1.3685 Accuracy 0.0043
Epoch 1 Batch 100 Loss 1.0085 Accuracy 0.0750
Epoch 1 Batch 200 Loss 0.9674 Accuracy 0.0867
Epoch 1 Batch 300 Loss 0.9443 Accuracy 0.1059
Epoch 1 Batch 344 Loss 0.9433 Accuracy 0.0930
Epoch 1 Loss 0.9980 Acc 0.0802
Time taken for 1 epoch 57.32 sec

Epoch 1 Batch 0 Val Loss 0.9103 Val Accuracy 0.0992
Epoch 1 Batch 33 Val Loss 0.8312 Val Accuracy 0.0918
Epoch 1 Val Loss 0.8589 Val Acc 0.0962
Time taken for 1 epoch 69.33 sec

Epoch 2 Batch 0 Loss 0.9165 Accuracy 0.1027
Epoch 2 Batch 100 Loss 0.8690 Accuracy 0.1090
Epoch 2 Batch 200 Loss 0.7102 Accuracy 0.1250
Epoch 2 Batch 300 Loss 0.7326 Accuracy 0.1527
Epoch 2 Batch 344 Loss 0.6521 Accuracy 0.1414
Epoch 2 Loss 0.7787 Acc 0.1256
Time taken for 1 epoch 16.46 sec

Epoch 2 Batch 0 Val Loss 0.6806 Val Accuracy 0.1504
Epoch 2 Batch 33 Val Loss 0.6658 Val Accuracy 0.1418
Epoch 2 Val Loss 0.657

In [ ]:
def sweep():

    wandb.init(config=default_config, magic=True, project="a3_collab", entity="cs21m002")
    config = wandb.config
    
    encoder, decoder = train(dataset, config, log_wandb)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="a3_collab", entity="cs21m002")

Create sweep with ID: oz1w8cse
Sweep URL: https://wandb.ai/cs21m002/a3_collab/sweeps/oz1w8cse


In [ ]:
wandb.agent(sweep_id, function=sweep, project="a3_collab", entity="cs21m002", count=50)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Epoch 10 Batch 100 Loss 0.2889 Accuracy 0.2477


wandb: Agent Starting Run: h55qqkya with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.1
wandb: 	embedding_dim: 64
wandb: 	epochs: 20
wandb: 	layer_type: LSTM
wandb: 	num_layers: 1
wandb: 	optimiser: nadam
wandb: 	units: 256


Epoch 10 Batch 200 Loss 0.3246 Accuracy 0.2453


wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


...

layer_type:LSTM units:64 embedding_dim:64 optimiser:nadam epochs:20 dropout:0 batch_size:128 num_layers:1 
Epoch 1 Batch 0 Loss 1.4674 Accuracy 0.5848
Epoch 1 Batch 100 Loss 1.0624 Accuracy 0.0574
Epoch 1 Batch 200 Loss 1.0292 Accuracy 0.0699
Epoch 1 Batch 300 Loss 0.9849 Accuracy 0.0820
Epoch 1 Batch 344 Loss 0.9637 Accuracy 0.0805
Epoch 1 Loss 1.0695 Acc 0.0703
Time taken for 1 epoch 32.53 sec

Epoch 1 Batch 0 Val Loss 0.9552 Val Accuracy 0.0730
Epoch 1 Batch 33 Val Loss 0.9432 Val Accuracy 0.0770
Epoch 1 Val Loss 0.9533 Val Acc 0.0788
Time taken for 1 epoch 39.28 sec

Epoch 2 Batch 0 Loss 0.9908 Accuracy 0.0855
Epoch 2 Batch 100 Loss 0.9840 Accuracy 0.0949
Epoch 2 Batch 200 Loss 0.9011 Accuracy 0.0934
Epoch 2 Batch 300 Loss 0.8460 Accuracy 0.1082
Epoch 2 Batch 344 Loss 0.8572 Accuracy 0.0977
Epoch 2 Loss 0.9220 Acc 0.0955
Time taken for 1 epoch 10.28 sec

Epoch 2 Batch 0 Val Loss 0.8237 Val Accuracy 0.1055
Epoch 2 Batch 33 Val Loss 0.8289 Val Accuracy 0.0941
Epoch 2 Val Loss 0.8249 

accuracy,▁▂▂▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇█▇█▇█▇██▇█
ep_training_accuracy,▁▂▃▃▄▄▅▆▆▆▇▇▇▇▇█████
ep_training_loss,█▆▅▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁
ep_val_accuracy,▁▂▃▃▄▄▅▆▆▇▇▇▇▇▇█████
ep_val_loss,█▆▅▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
loss,█▇▇▆▅▅▄▄▄▄▄▄▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁
val accuracy,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇████████████
val loss,██▇▇▆▆▅▅▄▄▅▅▄▄▃▃▃▃▂▂▃▃▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁
accuracy,0.22734
ep_training_accuracy,0.23249


wandb: Agent Starting Run: ypnl62nk with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0
wandb: 	embedding_dim: 64
wandb: 	epochs: 20
wandb: 	layer_type: GRU
wandb: 	num_layers: 2
wandb: 	optimiser: nadam
wandb: 	units: 256
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


layer_type:GRU units:256 embedding_dim:64 optimiser:nadam epochs:20 dropout:0 batch_size:128 num_layers:2 
Epoch 1 Batch 0 Loss 1.4610 Accuracy 0.0008
Epoch 1 Batch 100 Loss 1.0329 Accuracy 0.0770
Epoch 1 Batch 200 Loss 0.9768 Accuracy 0.0781
Epoch 1 Batch 300 Loss 0.9176 Accuracy 0.0957
Epoch 1 Batch 344 Loss 0.9114 Accuracy 0.0992
Epoch 1 Loss 1.0060 Acc 0.0786
Time taken for 1 epoch 54.65 sec

Epoch 1 Batch 0 Val Loss 0.8882 Val Accuracy 0.0898
Epoch 1 Batch 33 Val Loss 0.8804 Val Accuracy 0.0891
Epoch 1 Val Loss 0.8743 Val Acc 0.0929
Time taken for 1 epoch 65.82 sec

Epoch 2 Batch 0 Loss 0.8835 Accuracy 0.0973
Epoch 2 Batch 100 Loss 0.7997 Accuracy 0.1227
Epoch 2 Batch 200 Loss 0.7714 Accuracy 0.1281
Epoch 2 Batch 300 Loss 0.7428 Accuracy 0.1340
Epoch 2 Batch 344 Loss 0.6553 Accuracy 0.1465
Epoch 2 Loss 0.7851 Acc 0.1268
Time taken for 1 epoch 15.81 sec

Epoch 2 Batch 0 Val Loss 0.6298 Val Accuracy 0.1316
Epoch 2 Batch 33 Val Loss 0.6691 Val Accuracy 0.1344
Epoch 2 Val Loss 0.6752 

accuracy,▁▁▂▃▃▄▄▅▅▅▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇███▇██▇██████
ep_training_accuracy,▁▂▄▅▆▆▇▇▇▇▇█████████
ep_training_loss,█▆▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁
ep_val_accuracy,▁▃▅▆▇▇██████████████
ep_val_loss,█▆▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
loss,██▇▆▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val accuracy,▁▁▂▂▄▄▅▅▆▆▆▆▇▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇██▇▇▇▇████▇▇
val loss,██▆▆▅▅▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.32305
ep_training_accuracy,0.33051


In [ ]:
best_model_config = {
    "layer_type": "GRU",
    "units": 256,
    "embedding_dim": 64,
    "optimiser": "nadam",
    "epochs": 20,
    "dropout": 0.1,
    "batch_size": dataset.batch_size,
    "num_layers": 2
}

In [ ]:
enc, dec = train(dataset, best_model_config)

layer_type:GRU units:256 embedding_dim:64 optimiser:nadam epochs:20 dropout:0.1 batch_size:128 num_layers:2 
Epoch 1 Batch 0 Loss 1.4791 Accuracy 0.0027
Epoch 1 Batch 100 Loss 1.0421 Accuracy 0.0730
Epoch 1 Batch 200 Loss 0.9729 Accuracy 0.0844
Epoch 1 Batch 300 Loss 0.8729 Accuracy 0.0863
Epoch 1 Batch 344 Loss 0.9290 Accuracy 0.0953
Epoch 1 Loss 1.0122 Acc 0.0774
Time taken for 1 epoch 57.34 sec

Epoch 1 Batch 0 Val Loss 0.9271 Val Accuracy 0.0836
Epoch 1 Batch 33 Val Loss 0.9175 Val Accuracy 0.0820
Epoch 1 Val Loss 0.8800 Val Acc 0.0866
Time taken for 1 epoch 68.60 sec

Epoch 2 Batch 0 Loss 0.9048 Accuracy 0.0895
Epoch 2 Batch 100 Loss 0.8787 Accuracy 0.1113
Epoch 2 Batch 200 Loss 0.7959 Accuracy 0.1277
Epoch 2 Batch 300 Loss 0.8446 Accuracy 0.1305
Epoch 2 Batch 344 Loss 0.7722 Accuracy 0.1391
Epoch 2 Loss 0.8267 Acc 0.1143
Time taken for 1 epoch 16.04 sec

Epoch 2 Batch 0 Val Loss 0.7258 Val Accuracy 0.1254
Epoch 2 Batch 33 Val Loss 0.7193 Val Accuracy 0.1242
Epoch 2 Val Loss 0.725

In [ ]:
def evaluate(sentence, encoder, decoder):
    attention_plot = np.zeros((dataset.max_target_seq_length, dataset.max_input_seq_length))

    sentence = dataset.preprocess_word(sentence)

    inputs = [dataset.input_tokenizer.word_index[i] for i in sentence]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],maxlen=dataset.max_input_seq_length,padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    if encoder.layer_type != "LSTM":
        hidden = [tf.zeros((1, encoder.enc_units))]
    else:
        hidden = [tf.zeros((1, encoder.enc_units)), tf.zeros((1, encoder.enc_units))]
    
    enc_out, enc_hidden, enc_cell = encoder(inputs, hidden)
    
    if decoder.layer_type != "LSTM":
        dec_hidden = enc_hidden
    else:
        dec_hidden, dec_cell = enc_hidden, enc_cell

    dec_input = tf.expand_dims([dataset.target_tokenizer.word_index[sos]], 0)

    for t in range(dataset.max_target_seq_length):
        # passing enc_output to the decoder
        if decoder.layer_type != "LSTM":
            predictions, _, _ = decoder(dec_input, dec_hidden)
        else:
            predictions, _, _ = decoder(dec_input, dec_hidden, dec_cell)
                
        
        predicted_id = tf.argmax(predictions[0]).numpy()
        
        result += dataset.target_tokenizer.index_word[predicted_id]

        if dataset.target_tokenizer.index_word[predicted_id] == eos:
            return result, sentence

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [ ]:
def translate(sentence, encoder, decoder):
    result, sentence = evaluate(sentence, encoder, decoder)

    print('Input:', sentence)
    print('Predicted translation:', result)

In [ ]:
translate("abhishek", enc, dec)

Input: @abhishek#
Predicted translation: अभिषेक#


In [ ]:
wandb.init()

wandb: WARNING wandb.init() arguments ignored because wandb magic has already been initialized


In [ ]:
def save_predictions(data_frame, name):
    accuracy_count = 0;
    with open(name, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["INPUT", "PREDICTION", "TRUE"])
        for i, (inp, trg) in enumerate(zip(data_frame[1], data_frame[0])): 
            result, sentence = evaluate(inp, enc, dec)
            writer.writerow([inp, result[:-1], trg])
            print(inp, result[:-1], trg)
            if result[:-1] == trg:
                accuracy_count += 1
            if (i+1) % 100 == 0 or i+1 == data_frame.size:
                print("Accuracy", (accuracy_count / (i+1)))
                wandb.log({"test_accuracy": (accuracy_count / (i+1))})

    return accuracy_count/data_frame.size

In [ ]:
save_predictions(test_df, str(datetime.datetime.now().strftime("%b-%d-%Y-%H-%M-%S"))+"without-attn-predictions.csv")

ank अंक अंक
anka आंका अंक
ankit आंकित अंकित
anakon अनकों अंकों
ankhon आँ अंकों
ankon आं अंकों
angkor अंकर अंकोर
ankor अंकर अंकोर
angaarak अंगाक अंगारक
angarak अंगरक अंगारक
angraji अंग्रजी अंग्रज़ी
angreji अग्रेंजी अंग्रज़ी
angrzi अंग्रजी अंग्रज़ी
antah अंतांतांतांतांतांतां अंतः
antaha अंता अंतः
antarmukh अंशंशंशंशंशंशंशंशंशं अंतर्मुख
antmurkh अंचंचंचंचंचंचंचंचंचं अंतर्मुख
antrmukh अंशंशंशंशंशंशंशंशंशं अंतर्मुख
andrabee अंबी अंद्राबी
andrabi अंदी अंद्राबी
andha अंधा अंधा
andhapan अंधपन अंधापन
andheri अंधी अंधेरी
ambaani अमबानी अंबानी
ambani अमबानी अंबानी
akram अक्रम अकरम
acapulco एकॉकॉकॉकॉकॉकॉकॉकॉकॉक अकापुल्को
akapulko अको अकापुल्को
akhand अखंड अखंड
akhil आखिल अखिल
agavai अगवाई अगवाई
agvaai अग्ञाई अगवाई
augustine एन अगस्टाइन
augustustine आइन अगस्टाइन
agsta अग्ता अगस्ता
agusta आगुस्ता अगस्ता
augusta औगस्ता अगस्ता
agr एगर अग्र
agra आगरा अग्र
achambha अचंभा अचंभा
achanbha अपा अचंभा
achievement अचिमेंट अचीवमेंट
achivement अछिमेंट अचीवमेंट
acchi अच्छी अच्छी
achchi अच्छी अच्छी
achhi अछी अच्छी

0.058418480675255444